In [ ]:
from datetime import datetime
date=datetime.now()
year=date.year 
month=date.month 
day=1

my_date=datetime(year,month,day)


In [ ]:
type(year)

In [ ]:
my_date=my_date.strftime("%Y-%m-%d") 
my_date

In [ ]:
import pandas as pd
df = pd.read_csv(
        "../data/Hist_2023-03-20.csv", parse_dates=['日期'], index_col=0, dtype={"股票代码": object})
    
dates = df.index.unique().sort_values().to_list()
dates=[x.strftime("%Y-%m-%d") for x in dates]
dates[0]

In [ ]:
df.index

In [ ]:
df.loc["2023-03-06"]

In [ ]:
df.info()

In [ ]:
import pandas as pd
df = pd.read_csv(
    "../data/hist_2023-03-14.csv", parse_dates=['日期'], index_col=0, dtype={"股票编号": object})
df.info()

In [ ]:
import pandas as pd
value = pd.read_csv(
    "../data/总股本.csv", index_col=0, dtype={"代码": object})
value_dict = value['总股本'].to_dict()
value_dict['688018']
# value_dict


In [ ]:
df['总股本'] = df['股票代码'].apply(lambda x: value_dict.get(x))
df['总市值'] = df['总股本']*df['收盘']


In [ ]:
df.isnull().sum()

In [ ]:
# 按日期和板块名称分组，计算涨跌幅的平均值和成交量的总和
result = df.groupby(["日期", "板块名称"]).agg({"涨跌幅": "mean", "总市值": "sum"})
# 打印结果
print(result)


In [ ]:
df.index.value_counts()


In [ ]:
result=result.loc['2023-03-13']


In [ ]:
result.reset_index(inplace=True)
result

In [ ]:
import plotly.express as px
fig = px.treemap(result, path=[px.Constant('All'), '板块名称'], values='总市值', height=1080, width=1920,
                 color='涨跌幅', color_continuous_scale='Geyser', range_color=[-0.05, 0.05], color_continuous_midpoint=0,
                 hover_data={"总市值": ':,.2f', '涨跌幅':".2%"})
fig.show()

In [ ]:
result[(result['总市值'] >= int(10)*100_000_000)]


In [ ]:
result[(result['总市值'] <= int(200)*100_000_000)]


## 获得总股本

In [ ]:
url = "http://push2his.eastmoney.com/api/qt/stock/kline/get"
params = {
    "fields1": "f1,f2,f3,f4,f5,f6",
    "fields2": "f51,f52,f53,f54,f55,f56,f57,f58,f59,f60,f61,f116",
    "ut": "7eea3edcaed734bea9cbfc24409ed989",
    "klt": "101",
    "fqt": "0",
    "secid": "1.000001",
    "beg": "20230301",
    "end": "20230401",
    "_": "1623766962675",
}
import requests
r = requests.get(url, params=params)
print(r.text)

In [ ]:
from akshare.stock_feature.stock_hist_em import code_id_map_em
code_id_dict = code_id_map_em()
# code_id_dict['300119']

In [ ]:
df = pd.Series(code_id_dict).to_frame('side')
df.index.name="code"
df

In [ ]:
# df = pd.DataFrame(code_id_dict)
df.to_csv("../data/id_code.csv", header=True)


In [ ]:
df.head()

In [ ]:
code_id_dict['300119']


In [ ]:
code_df = pd.read_csv("../data/板块名称_股票对应.csv", index_col=0)
code_dict = code_df.set_index("代码")['板块名称'].to_dict()
code_dict


In [ ]:
code_df = pd.read_csv("../data/id_code.csv", index_col=0,dtype={"code":object})

code_df.info()

In [ ]:
code_df["side"].to_dict()

## 资金流入函数

In [ ]:
import akshare as ak

stock_individual_fund_flow_df = ak.stock_individual_fund_flow(
    stock="600094", market="sh")
print(stock_individual_fund_flow_df)


In [ ]:
stock_individual_fund_flow_df.info()

## 均线函数

In [ ]:
import akshare as ak

stock_rank_xstp_ths_df = ak.stock_rank_xstp_ths(symbol="5日均线")
print(stock_rank_xstp_ths_df)


In [ ]:
stock_rank_xstp_ths_df.info()

In [ ]:
stock_sector_fund_flow_rank_df = ak.stock_sector_fund_flow_rank(
    indicator="5日", sector_type="行业资金流")
print(stock_sector_fund_flow_rank_df)


In [ ]:
import pandas as pd
from datetime import datetime
df = pd.read_csv(
    f"../data/result_100-500_{datetime.now().strftime('%Y%m%d')}.csv", parse_dates=['日期'], index_col=0, dtype={"股票代码": object})
df=df.loc['2023-03-01']

# 其他传媒


In [ ]:
df[df['板块名称'] == "其他传媒"]


In [ ]:
mydf = pd.read_csv(
    f"../data/merge_{datetime.now().strftime('%Y-%m-%d')}.csv", parse_dates=['日期'], index_col=0, dtype={"股票代码": object})
mydf=mydf.loc['2023-03-01']
mydf.head()

In [ ]:
mydata = mydf[mydf['板块名称'] == "其他传媒"]
mydata.info()


In [ ]:
mydata

In [ ]:
result = mydata.groupby(['日期', '板块名称'])['涨跌幅'].agg(
    [('涨的数量', lambda x: sum(x > 0)), ('跌的数量', lambda x: sum(x < 0)), ('平盘', lambda x: sum(x == 0))])
result

In [ ]:
bins = [-20, -10, -5, -3, -0.099, 0.099, 3, 5, 10, 20]
# bins = list(range(-11, 12))
cuts = pd.cut(mydata['涨跌幅'], bins=bins)
pct_chg_list = mydata.groupby([cuts])['涨跌幅'].count()
# cur_df = pct_chg_list.unstack()
pct_chg_list 
